In [16]:
import numpy as np
import uproot

In [17]:
def getRot( apparent, actual):
    """
    computes rotation from A to B when rotated through origin.
    shift A and B before, if rotation did not already occur through origin!

    see https://math.stackexchange.com/a/476311
    or https://en.wikipedia.org/wiki/Cross_product
    and https://en.wikipedia.org/wiki/Rotation_matrix#Conversion_from_rotation_matrix_and_to_axis%E2%80%93angle

    This function works on 3D points only, do not give homogeneous coordinates to this!
    """
    # error handling
    if np.linalg.norm(apparent) == 0 or np.linalg.norm(actual) == 0:
        self.logger.log("\nERROR. can't create rotation with null vector!\n")
        return

    # assert shapes
    assert apparent.shape == actual.shape

    # normalize vectors
    apparent = apparent / np.linalg.norm(apparent)
    actual = actual / np.linalg.norm(actual)

    # calc rot angle by dot product
    cosine = np.dot(apparent, actual)  # cosine

    # make 2D vectors so that transposing works
    cVector = apparent[np.newaxis].T
    dVector = actual[np.newaxis].T

    # compute skew symmetric cross product matrix
    crossMatrix = (dVector @ cVector.T) - (cVector @ dVector.T)

    # compute rotation matrix
    # TODO wait, shouldn't there be a sine somewhere here? check that 1/1+cos again!
    R = (
        np.identity(3)
        + crossMatrix
        + np.dot(crossMatrix, crossMatrix) * (1 / (1 + cosine))
    )

    return R

def rotationMatrixToEulerAngles(R):

    assert R.shape == (4, 4) or R.shape == (3, 3)

    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6

    if not singular:
        x = np.arctan2(R[2, 1], R[2, 2])
        y = np.arctan2(-R[2, 0], sy)
        z = np.arctan2(R[1, 0], R[0, 0])
    else:
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])


In [18]:
from pathlib import Path
import json

def loadMatrices(fileName, reshape=True):
    if fileName is None:
        raise Exception(f"ERROR! Filename is None, something went wrong!")
    if not Path(fileName).exists():
        raise Exception(f"ERROR! File not found: {fileName}!")

    with open(fileName, "r") as f:
        temp = json.load(f)

    if reshape:
        for p in temp:
            temp[p] = np.array(temp[p]).reshape(4, 4)
    return temp


def saveMatrices(matrices, fileName):

    # create path if needed
    if not Path(fileName).parent.exists():
        Path(fileName).parent.mkdir()

    # warn if overwriting
    if Path(fileName).exists():
        print(f"WARNING. Replacing file: {fileName}!\n")
        Path(fileName).unlink()

    # flatten matrices, make a copy (pass-by-reference!)
    saveMatrices = {}
    for p in matrices:
        saveMatrices[p] = np.ndarray.tolist(np.ndarray.flatten(matrices[p]))

    with open(fileName, "w") as f:
        json.dump(saveMatrices, f, indent=2, sort_keys=True)

In [19]:
def quantileCut(xyzArray, cut=4):

    if cut == 0:
        return xyzArray

    # calculate cut length
    cut = int(len(xyzArray) * (cut / 100))

    # calculate center of mass (where most points are)
    # don't use average, some values are far too large, median is a better estimation
    comMed = np.median(xyzArray[:, 3:6], axis=0)

    # now, sort by distance and cut largest
    # first, calculate distace of each point to center of mass
    distVec = xyzArray[:, 3:6] - comMed

    # calculate length of these distance vectors
    distVecNorm = np.linalg.norm(distVec, axis=1)

    # sort the entire array by this length
    xyzArray = xyzArray[distVecNorm.argsort()]

    # cut the largest values
    resxyzArrayxyzArray = xyzArray[:-cut]

    return resxyzArrayxyzArray


def getIPfromMultiple(filename, maxNoOfFiles=0):
    # fileTree = uproot.open(filename)['pndsim']

    # make empty 2D (n times 4) result array for each individual IP position (that's per file)
    IPs = np.empty((0, 4))

    runIndex = 0
    for array in uproot.iterate(
        filename,
        [
            "LMDTrackQ.fTrkRecStatus",
            "LMDTrackQ.fXrec",
            "LMDTrackQ.fYrec",
            "LMDTrackQ.fZrec",
        ],
        library="np",
    ):

        recStat = np.concatenate(array["LMDTrackQ.fTrkRecStatus"]).ravel()
        recX = np.concatenate(array["LMDTrackQ.fXrec"]).ravel()
        recY = np.concatenate(array["LMDTrackQ.fYrec"]).ravel()
        recZ = np.concatenate(array["LMDTrackQ.fZrec"]).ravel()

        # apply mask for correctly reconstructed track and tracks within 5cm
        # that means reconstructed IP must be within 5cm of 0 in all directions
        thresh = 5
        mask = (
            (recStat == 0)
            & (np.abs(recX) < thresh)
            & (np.abs(recY) < thresh)
            & (np.abs(recZ) < thresh)
        )

        recXmask = recX[mask]
        recYmask = recY[mask]
        recZmask = recZ[mask]

        # don't worry, this is done by reference, nothing is copied here
        outarray = np.array([recXmask, recYmask, recZmask]).T

        outarray = quantileCut(outarray, 4)

        foundIP = np.average(outarray, axis=0)
        resultIPhomogeneous = np.ones(4)
        resultIPhomogeneous[:3] = foundIP
        # print(f"loaded {len(outarray)} tracks")
        # print(f"found ip: {resultIPhomogeneous}")
        IPs = np.vstack((IPs, resultIPhomogeneous))
        runIndex += 1
        if runIndex == maxNoOfFiles:
            break

    print(f"read {runIndex} file(s)")
    return np.average(IPs, axis=0)


In [20]:
np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

idealDetectorMatrixPath = "input/backup/detectorMatricesIdeal.json"
idealDetectorMatrices = loadMatrices(idealDetectorMatrixPath)

# ipActual = np.array([-0.023392, 0.00718, 0.004023, 1.0])

rootFileWildcard = "comparisonData/box100/rawData/Lumi_TrksQA_*.root:pndsim"
ipMyFunc = getIPfromMultiple(rootFileWildcard,1)
print(f'found this ip: {ipMyFunc}')
ipStefanFit = [-0.12169, -0.107023, 0, 1]
ipApparent = ipMyFunc
print(f'will be using this ip: {ipApparent}')


# we want the rotation of the lumi box, so we have to change the basis
matPndtoLmd = idealDetectorMatrices["/cave_1/lmd_root_0"]
zero = [0, 0, 0, 1]

zeroAt = (np.linalg.inv(matPndtoLmd) @ zero)[:3]
ipApparentLMD = (np.linalg.inv(matPndtoLmd) @ ipApparent)[:3]

print(f"zero at: {zeroAt}\nipApparent: {ipApparentLMD}")

#! order is: IP_from_LMD, IP_actual (i.e. from PANDA)
Ra = getRot(ipApparentLMD, zeroAt)

# homogenize the matrix again
R1 = np.identity(4)
R1[:3, :3] = Ra

# homogenize the matrix again
R1 = np.identity(4)
R1[:3, :3] = Ra

# after that, add the remaining translation (direct shift towards IP), not implemented yet
resultJson = {"/cave_1/lmd_root_0": R1}
# print('R1',R1)
# print('----------------')

earlierMatrix = loadMatrices("comparisonData/box100/alMat-IPalignment-1.00.json")[
    "/cave_1/lmd_root_0"
]

# print('earlierMatrix', earlierMatrix)
# print('----------------')
# print('residualsMatrix:')
mRes = R1 @ np.linalg.inv(earlierMatrix)
# print(mRes)
# print('----------------')
# print('Design Misalignment Matrix:')
mMis = loadMatrices("comparisonData/misMatrices/misMat-box100-1.00.json")[
    "/cave_1/lmd_root_0"
]
# print(mMis)
print("----------------")
print("Euler Angles of mMis:")
print(rotationMatrixToEulerAngles(mMis))
print("----------------")

print("Euler Angles of earlierMatrix:")
print(rotationMatrixToEulerAngles(earlierMatrix))
print("----------------")

print("Euler Angles of R1:")
print(rotationMatrixToEulerAngles(R1))


read 1 file(s)
found this ip: [-0.128766 -0.105278 -0.000059  1.      ]
will be using this ip: [-0.128766 -0.105278 -0.000059  1.      ]
zero at: [   19.018047     0.       -1109.257283]
ipApparent: [   18.889386    -0.105278 -1109.262494]
----------------
Euler Angles of mMis:
[ 0.0001 -0.0001  0.0001]
----------------
Euler Angles of earlierMatrix:
[ 0.000094 -0.000129  0.000005]
----------------
Euler Angles of R1:
[ 0.000095 -0.000116  0.000002]
